### 1.baseline与coco-model 在seed0下的图片比较（均与原始真实图像相比）
#### 1.1 组间相似度

In [12]:
import json
img_text_dic = {}
with open("/home/cl/CV_Proj/evaluate/metadata_test.jsonl") as fin:
    for line in fin.readlines():
        tmp_dic = json.loads(line)
        img_text_dic[tmp_dic["file_name"]] = tmp_dic["text"]
        print(tmp_dic["file_name"],tmp_dic["text"])

000000000139_0.jpg A woman stands in the dining area at the table.
000000000139_1.jpg A room with chairs, a table, and a woman in it.
000000000139_2.jpg A woman standing in a kitchen by a window
000000000139_3.jpg A person standing at a table in a room.
000000000139_4.jpg A living area with a television and a table
000000000285_0.jpg A big burly grizzly bear is show with grass in the background.
000000000285_1.jpg The large brown bear has a black nose.
000000000285_2.jpg Closeup of a brown bear sitting in a grassy area.
000000000285_3.jpg A large bear that is sitting on grass. 
000000000285_4.jpg A close up picture of a brown bear's face.
000000000632_0.jpg Bedroom scene with a bookcase, blue comforter and window.
000000000632_1.jpg A bedroom with a bookshelf full of books.
000000000632_2.jpg This room has a bed with blue sheets and a large bookcase
000000000632_3.jpg A bed and a mirror in a small room.
000000000632_4.jpg a bed room with a neatly made bed a window and a book shelf
0000

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import clip


In [4]:
model, preprocess = clip.load('ViT-B/32')

model.cuda().eval()

100%|███████████████████████████████████████| 338M/338M [05:17<00:00, 1.12MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [37]:
# image_path = 'flickr30k/flickr30k-images'
base_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/baseline_inference'
coco_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/coco-model_inference'
ori_img_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/original_imgs_5003'

ori_images = []
text = []

for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(ori_img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    ori_images.append(cur_image)
    
    cur_text = [img_text_dic[item]]
    text.append(cur_text)

In [38]:
coco_images = []
base_images = []

for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(coco_img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    coco_images.append(cur_image)
    
for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(base_img_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    base_images.append(cur_image)   

In [41]:
# base_img_inputs = torch.stack(base_images, dim=0).cuda()
# coco_img_inputs = torch.stack(coco_images, dim=0).cuda()
ori_img_inputs = torch.stack(ori_images, dim=0).cuda()

with torch.no_grad():
#     base_img_features = model.encode_image(base_img_inputs)
#     coco_img_features = model.encode_image(coco_img_inputs)
    ori_img_features = model.encode_image(ori_img_inputs)

In [8]:
total_cnt = 0
text_features = []
real_labels = []
for idx, sent_list in enumerate(tqdm(texts, desc='Encoding sentences')):
    try:
        text_input = clip.tokenize(sent_list).cuda()
    except Exception as e:
        print(str(e))
        continue

    total_cnt += len(sent_list)
    real_labels += [idx for _ in range(len(sent_list))]

    with torch.no_grad():
        text_feature = model.encode_text(text_input)

    text_features.append(text_feature)

text_features = torch.cat(text_features, dim=0)

Encoding sentences: 100%|███████████████████████████████████████████████████████████| 5003/5003 [00:50<00:00, 99.32it/s]


In [5]:
def sim_fnt(x, y):
    return F.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0), dim=2)

In [45]:
#查看baseline与coco model生成图片分别和原始真实图片的余弦相似度
base_ori_sim = sim_fnt(base_img_features, ori_img_features).mean()
coco_ori_sim = sim_fnt(coco_img_features, ori_img_features).mean()
print("The similarity between baseline images and original images is:",base_ori_sim.item())
print("The similarity between coco-model images and original images is:",coco_ori_sim.item())

The similarity between baseline images and original images is: 0.712890625
The similarity between coco-model images and original images is: 0.73876953125




——————————————————————————————————————————————————————————————————————
#### 1.2 组内相似度

以baseline为例，每个真实图片对应5或6句话，生成5或6张图，称为1个group，计算这1个group中两两相似度后，再对整个组的多个group求均值，得到组内相似度

In [8]:
#定义一组5个图片组内比较相似度
def cal_group_sim(features):
    return F.cosine_similarity(features.unsqueeze(0), features.unsqueeze(1), dim=-1).mean()
    
def cal_ave_group_sim(features):
    cur = ''
    ori = '000000000139'
    group = [] #作为base_img_features的index
    total_sim = []
    total_num = 0
    for i,key in enumerate(tqdm(img_text_dic.keys(),desc='Computing Similarity')):
        cur = key[0:12]
        if(cur == ori):
            group.append(i)
        else:
            total_num += 1
            ori = cur
#             print("group is:",group)
            total_sim.append(cal_group_sim(features[group]))
#             print(features[group])
            group=[]
            

    ave_sim = (sum(total_sim)/len(total_sim)).item()
    max_sim = (max(total_sim)).item()
    min_sim = (min(total_sim)).item()
    total_sim_int = [i.cpu() for i in total_sim]
    std_sim = (np.std(total_sim_int))
                           
    print("The Within-group Average Similarity is:",round(ave_sim * 100,2))
    print("The Within-group Max Similarity is:",round(max_sim * 100,2))
    print("The Within-group Min Similarity is:",round(min_sim * 100,2))
    print("The Standard Deviation of Within-group similarity is:",round(std_sim * 100,2))

In [39]:
#baseline一组5（或6）个图片组内比较相似度
print("The baseline group:")
cal_ave_group_sim(base_img_features)

The baseline group:


NameError: name 'base_img_features' is not defined

In [ ]:
#coco一组5（或6）个图片组内比较相似度
print("The coco model group:")
cal_ave_group_sim(coco_img_features)

In [ ]:
#原始图片一组5（或6）个图片组内比较相似度
print("The original group:")
cal_ave_group_sim(ori_img_features)

——————————————————————————————————————————————————————————————————————

### 2.coco-model-bt、coco-model-crop_swap、linear-p-b1-n10共3个 model生成的图片与原始真实图片相比
#### 2.1 coco-model-bt、coco-model-crop_swap、linear-p-b1-n10组间相似度

In [13]:
# image_path = 'flickr30k/flickr30k-images'
bt_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/coco-model-bt_inference'
crop_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-crop_swap_inference'
b1n10_path = '/home/cl/CV_Proj/evaluate/text2img2/text2img/coco-model-linear_p_beta1_n5_inference'



bt_img = []
crop_img = []
b1n10_img = []

for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(bt_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    bt_img.append(cur_image)

for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(crop_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    crop_img.append(cur_image)

for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(b1n10_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    b1n10_img.append(cur_image)

In [14]:
bt_img_inputs = torch.stack(bt_img, dim=0).cuda()
crop_img_inputs  = torch.stack(crop_img, dim=0).cuda()
b1n10_img_inputs  = torch.stack(b1n10_img, dim=0).cuda()

with torch.no_grad():
    bt_img_features = model.encode_image(bt_img_inputs)
    crop_img_features = model.encode_image(crop_img_inputs)
    b1n10_img_features = model.encode_image(b1n10_img_inputs)

In [15]:
#查看3个模型生成的图片分别和原始真实图片的余弦相似度
bt_ori_sim = sim_fnt(bt_img_features, ori_img_features).mean()
crop_ori_sim = sim_fnt(crop_img_features, ori_img_features).mean()
b1n10_ori_sim = sim_fnt(b1n10_img_features, ori_img_features).mean()
print("The similarity between bt-model images and original images is:",bt_ori_sim.item())
print("The similarity between crop-model images and original images is:",crop_ori_sim.item())
print("The similarity between b1n10-model images and original images is:",b1n10_ori_sim.item())

NameError: name 'ori_img_features' is not defined



——————————————————————————————————————————————————————————————————————
#### 2.2 coco-model-bt、coco-model-crop_swap、linear-p-b1-n10组内相似度


In [16]:
# #原始图片一组5（或6）个图片组内比较相似度
print("The  group:")
cal_ave_group_sim(ori_img_features)

The  group:


NameError: name 'ori_img_features' is not defined

In [17]:
#baseline一组5（或6）个图片组内比较相似度
print("The coco-model-pt group:")
cal_ave_group_sim(bt_img_features)

The coco-model-pt group:


Computing Similarity: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 27413.68it/s]


The Within-group Average Similarity is: 85.94
The Within-group Max Similarity is: 95.9
The Within-group Min Similarity is: 58.89
The Standard Deviation of Within-group similarity is: 6.48


In [18]:
#原始coco一组5（或6）个图片组内比较相似度
print("The crop_swap-model group:")
cal_ave_group_sim(crop_img_features)

The crop_swap-model group:


Computing Similarity: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 24585.22it/s]

The Within-group Average Similarity is: 85.25
The Within-group Max Similarity is: 95.36
The Within-group Min Similarity is: 59.91
The Standard Deviation of Within-group similarity is: 6.42


In [19]:
#coco-model-bt一组5（或6）个图片组内比较相似度
print("The coco-model-bt group:")
cal_ave_group_sim(bt_img_features)

The coco-model-bt group:


Computing Similarity: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 40282.93it/s]

The Within-group Average Similarity is: 85.94
The Within-group Max Similarity is: 95.9
The Within-group Min Similarity is: 58.89
The Standard Deviation of Within-group similarity is: 6.48


In [20]:
#coco-model-crop_swap一组5（或6）个图片组内比较相似度
print("The coco-model-crop_swap group:")
cal_ave_group_sim(crop_img_features)

The coco-model-crop_swap group:


Computing Similarity: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 41196.84it/s]

The Within-group Average Similarity is: 85.25
The Within-group Max Similarity is: 95.36
The Within-group Min Similarity is: 59.91
The Standard Deviation of Within-group similarity is: 6.42


In [21]:
#coco-model-b1n10一组5（或6）个图片组内比较相似度
print("The coco-model-b1n10 group:")
cal_ave_group_sim(b1n10_img_features)

The coco-model-b1n10 group:


Computing Similarity: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 41085.69it/s]

The Within-group Average Similarity is: 85.06
The Within-group Max Similarity is: 95.07
The Within-group Min Similarity is: 58.2
The Standard Deviation of Within-group similarity is: 6.66


——————————————————————————————————————————————————————————————————————

### 3.linear-p-b1-n5、linear-p-b4-n5、linear-p-b8-n5共3个 model生成的图片与原始真实图片相比
#### 3.1 组间相似度

In [44]:
# image_path = 'flickr30k/flickr30k-images'
b1n5_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-linear_p_beta1_n5_inference'
b4n5_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-linear_p_beta4_n5_inference'
b8n5_path = '/home/cl/CV_Proj/evaluate/text2img_/coco-model-linear_p_beta8_n5_inference'


b1n5_img = []
b4n5_img = []
b8n5_img = []
   
for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(b1n5_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    b1n5_img.append(cur_image)
    
for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(b4n5_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    b4n5_img.append(cur_image)

for i, item in enumerate(img_text_dic):
    cur_image = Image.open(os.path.join(b8n5_path, item)).convert('RGB')
    cur_image = preprocess(cur_image)
    b8n5_img.append(cur_image)

In [45]:
b1n5_img_inputs  = torch.stack(b1n10_img, dim=0).cuda()
b4n5_img_inputs  = torch.stack(b4n5_img, dim=0).cuda()
b8n5_img_inputs  = torch.stack(b8n5_img, dim=0).cuda()

with torch.no_grad():
    b1n5_img_features = model.encode_image(b1n5_img_inputs)
    b4n5_img_features = model.encode_image(b4n5_img_inputs)
    b8n5_img_features = model.encode_image(b8n5_img_inputs)

In [47]:
#查看3个模型生成的图片分别和原始真实图片的余弦相似度
b1n5_ori_sim = sim_fnt(b1n5_img_features, ori_img_features).mean()
b4n5_ori_sim = sim_fnt(b4n5_img_features, ori_img_features).mean()
bt_ori_sim = sim_fnt(b8n5_img_features, ori_img_features).mean()

print("The similarity between b1n5-model images and original images is:",b1n5_ori_sim.item())
print("The similarity between b4n5-model images and original images is:",b4n5_ori_sim.item())
print("The similarity between b8n5-model images and original images is:",b8n5_ori_sim.item())

The similarity between b1n5-model images and original images is: 0.73779296875
The similarity between b4n5-model images and original images is: 0.73681640625
The similarity between b8n5-model images and original images is: 0.73681640625




——————————————————————————————————————————————————————————————————————
#### 3.2 linear-p-b1-n5、linear-p-b4-n5、linear-p-b8-n5组内相似度


In [42]:
#原始图片一组5（或6）个图片组内比较相似度
print("The  group:")
cal_ave_group_sim(ori_img_features)

The  group:


Computing Similarity: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 39291.66it/s]

The Within-group Average Similarity is: 100.0
The Within-group Max Similarity is: 100.0
The Within-group Min Similarity is: 100.0
The Standard Deviation of Within-group similarity is: 0.0


In [32]:
#baseline一组5（或6）个图片组内比较相似度
print("The coco-model-pt group:")
cal_ave_group_sim(bt_img_features)

The coco-model-pt group:


Computing Similarity: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 41471.38it/s]

The Within-group Average Similarity is: 85.94
The Within-group Max Similarity is: 95.9
The Within-group Min Similarity is: 58.89
The Standard Deviation of Within-group similarity is: 6.48


In [34]:
#原始coco一组5（或6）个图片组内比较相似度
print("The crop_swap-model group:")
cal_ave_group_sim(crop_img_features)

The crop_swap-model group:


Computing Similarity: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 41702.97it/s]

The Within-group Average Similarity is: 85.25
The Within-group Max Similarity is: 95.36
The Within-group Min Similarity is: 59.91
The Standard Deviation of Within-group similarity is: 6.42


In [31]:
#coco-model-b1n5一组5（或6）个图片组内比较相似度
print("The coco-model-b1n5 group:")
cal_ave_group_sim(b1n5_img_features)

The coco-model-b1n5 group:


Computing Similarity: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 40839.40it/s]

The Within-group Average Similarity is: 85.64
The Within-group Max Similarity is: 94.92
The Within-group Min Similarity is: 60.11
The Standard Deviation of Within-group similarity is: 6.2


In [28]:
#coco-model-b4n5一组5（或6）个图片组内比较相似度
print("The coco-model-b4n5 group:")
cal_ave_group_sim(b4n5_img_features)

The coco-model-b4n5 group:


Computing Similarity: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 41498.77it/s]

The Within-group Average Similarity is: 85.94
The Within-group Max Similarity is: 96.09
The Within-group Min Similarity is: 60.89
The Standard Deviation of Within-group similarity is: 6.29


In [29]:
#coco-model-b8n5一组5（或6）个图片组内比较相似度
print("The coco-model-b8n5 group:")
cal_ave_group_sim(b8n5_img_features)

The coco-model-b8n5 group:


Computing Similarity: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5003/5003 [00:00<00:00, 39641.86it/s]

The Within-group Average Similarity is: 85.6
The Within-group Max Similarity is: 95.31
The Within-group Min Similarity is: 57.28
The Standard Deviation of Within-group similarity is: 6.37
